In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime

# ================================
# تنظیمات عمومی — تارگت جدید
# ================================
TARGET_COL = 'AssetID_8341'                          # ← تارگت جدید
INPUT_FILE  = r"C:\BI\lube_oil_system_data_g11.xlsx"
OUTPUT_FILE = fr"C:\BI\regression_output_for_assetid_8341.xlsx"  # ← اسم فایل خودکار عوض شد
LOG_FILE    = r"C:\BI\regression_assetid_8341_log.txt"

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log(f"شروع پیش‌بینی {TARGET_COL} — با RecordDate + RecordTime + id + حفظ ترتیب کامل")

# ================================
# 1. خواندن فایل اصلی
# ================================
if not os.path.exists(INPUT_FILE):
    log(f"خطا: فایل ورودی پیدا نشد → {INPUT_FILE}")
    exit()

df_original = pd.read_excel(INPUT_FILE)
log(f"فایل اصلی خوانده شد → {len(df_original):,} ردیف")

# ================================
# 2. آماده‌سازی داده برای مدل
# ================================
non_feature_cols = ['id', 'RecordDate', 'RecordTime']
model_df = df_original.drop(columns=[c for c in non_feature_cols if c in df_original.columns], errors='ignore')

if TARGET_COL not in model_df.columns:
    log(f"خطا: ستون {TARGET_COL} در داده وجود ندارد!")
    exit()

X = model_df.drop(TARGET_COL, axis=1)
y = model_df[TARGET_COL]

log(f"تارگت: {TARGET_COL} | تعداد فیچرها: {X.shape[1]}")

# ================================
# 3. نرمالایز کردن
# ================================
log("نرمالایز کردن فیچرها با StandardScaler...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

# ================================
# 4. تقسیم داده و آموزش مدل
# ================================
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

log("در حال آموزش XGBoost...")
model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)
model.fit(X_train, y_train)
log("آموزش XGBoost با موفقیت انجام شد.")

# ================================
# 5. ارزیابی مدل
# ================================
y_pred_test = model.predict(X_test)
mae  = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2   = r2_score(y_test, y_pred_test)
mape = np.mean(np.abs((y_test - y_pred_test) / (y_test + 1e-8))) * 100

log("\n" + "="*60)
log(f"           نتایج ارزیابی مدل برای {TARGET_COL}")
log("="*60)
log(f"   MAE           : {mae:.8f}")
log(f"   RMSE          : {rmse:.8f}")
log(f"   R²            : {r2:.6f}")
log(f"   MAPE (%)      : {mape:.4f}%")
log("="*60)

# ================================
# 6. پیش‌بینی روی کل داده
# ================================
y_pred_full = model.predict(X_scaled_df)

# ================================
# 7. ساخت خروجی نهایی
# ================================
result = df_original.copy()

# ستون‌های پیش‌بینی با نام دینامیک
pred_col = f"{TARGET_COL}_Predicted"
error_col = "Error (Real - Predicted)"
abs_error_col = "Absolute_Error"
perc_error_col = "Percentage_Error (%)"

result[pred_col] = y_pred_full
result[error_col] = result[TARGET_COL] - y_pred_full
result[abs_error_col] = np.abs(result[error_col])
result[perc_error_col] = np.abs(result[error_col] / (result[TARGET_COL] + 1e-8)) * 100

# ================================
# 8. ترتیب ستون‌ها — حرفه‌ای و مرتب
# ================================
priority_cols = []

if 'RecordDate' in result.columns: priority_cols.append('RecordDate')
if 'RecordTime' in result.columns: priority_cols.append('RecordTime')
if 'id' in result.columns:         priority_cols.append('id')

priority_cols += [
    TARGET_COL,
    pred_col,
    error_col,
    abs_error_col,
    perc_error_col
]

remaining_cols = [col for col in result.columns if col not in priority_cols]
final_columns = priority_cols + remaining_cols

result = result[final_columns]

# ================================
# 9. ذخیره نهایی
# ================================
result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')

log(f"\nموفقیت‌آمیز! پیش‌بینی {TARGET_COL} کامل شد.")
log(f"فایل خروجی: {OUTPUT_FILE}")
log(f"ستون‌های اصلی: RecordDate → RecordTime → id → {TARGET_COL} → {pred_col} → خطاها...")

print("\n" + "="*80)
print(f"            پیش‌بینی {TARGET_COL} با موفقیت انجام شد!")
print("="*80)
print(f"   فایل خروجی → {OUTPUT_FILE}")
print(f"   تعداد ردیف  → {len(result):,}")
print(f"   MAE          → {mae:.8f}")
print(f"   MAPE         → {mape:.4f}%")
print(f"   R²           → {r2:.6f}")
print(f"   تارگت       → {TARGET_COL}")
print("="*80)
print("   حالا فایل رو باز کن — همه چیز مرتب و آماده آنالیز و داشبورد!")
print("="*80)

شروع پیش‌بینی AssetID_8341 — با RecordDate + RecordTime + id + حفظ ترتیب کامل
فایل اصلی خوانده شد → 3,000 ردیف
تارگت: AssetID_8341 | تعداد فیچرها: 6
نرمالایز کردن فیچرها با StandardScaler...
در حال آموزش XGBoost...
آموزش XGBoost با موفقیت انجام شد.

           نتایج ارزیابی مدل برای AssetID_8341
   MAE           : 0.06887802
   RMSE          : 0.09620392
   R²            : 0.296464
   MAPE (%)      : 55.8795%

موفقیت‌آمیز! پیش‌بینی AssetID_8341 کامل شد.
فایل خروجی: C:\BI\regression_output_for_assetid_8341.xlsx
ستون‌های اصلی: RecordDate → RecordTime → id → AssetID_8341 → AssetID_8341_Predicted → خطاها...

            پیش‌بینی AssetID_8341 با موفقیت انجام شد!
   فایل خروجی → C:\BI\regression_output_for_assetid_8341.xlsx
   تعداد ردیف  → 3,000
   MAE          → 0.06887802
   MAPE         → 55.8795%
   R²           → 0.296464
   تارگت       → AssetID_8341
   حالا فایل رو باز کن — همه چیز مرتب و آماده آنالیز و داشبورد!
